# Simple Layer and CI Operations

In [7]:
# import basic functions
from omnikeeper_client.functions import (
    get_access_token,
    create_graphql_client,
    create_layer,
    upsert_layerdata,
    hexString2RGBColor,
    mutate_cis,
    get_ci_attributes,
    build_graphQL_InsertCIAttributeInputType,
    create_ci,
    truncate_layer
)
import os

create a omnikeeper client

**TODO**: bring access token to request level instead of only static at creation - this will fail for long running apps

In [8]:
config = dict(
    omnikeeper_url=os.getenv('OMNIKEEPER_URL'),
    client_id=os.getenv('OMNIKEEPER_AUTH_CLIENTID'),
    username=os.getenv('OMNIKEEPER_AUTH_USERNAME'),
    password=os.getenv('OMNIKEEPER_AUTH_PASSWORD'),
)
access_token = get_access_token(config)
client = create_graphql_client("%s/graphql" % config['omnikeeper_url'], access_token)


/tmp/ipykernel_14735/1603544891.py:8: FutureWarning: Call to deprecated function (or staticmethod) create_graphql_client. (please use xyz instead) -- Deprecated since version 3.5.0.
  client = create_graphql_client("%s/graphql" % config['omnikeeper_url'], access_token)


now create a layer for holding our attributes:
- create/name it
- set description and color it
- clear all data from layer (if it already exists)

those steps are idempotent since we use upsert and create_layer does not throw an error if it already existed


In [9]:
target_layer = "testlayer"

# create layer, if it does not exist and set layer data
create_layer(client, target_layer)
upsert_layerdata(client, target_layer, "description", hexString2RGBColor("#353973"))

# empty layer, if anything is inside
truncate_layer(client, target_layer)


True

create a named CI and store attributes to it

In [10]:
# create a single CI with a name attribute
ciid1 = create_ci(client, "test-ci01", target_layer)

# insert an attribute
mutate_cis(client, target_layer, [target_layer], [build_graphQL_InsertCIAttributeInputType(ciid1, "test_attribute_1", "test_value_1")])

print (f"CI id={ciid1} created")


CI id=cdcf4c3d-ae1b-4216-9119-81eb26dd12f3 created


This CI should appear now in technical interface, mutate it there and check your modified attributes using this code:

In [11]:
# read CI
ci_attributes = get_ci_attributes(client, [target_layer], [ciid1])

print(ci_attributes)

{'cdcf4c3d-ae1b-4216-9119-81eb26dd12f3': {'__name': 'test-ci01', 'test_attribute_1': 'test_value_1'}}
